In [1]:
import pandas as pd
import numpy as np
import json
import pickle
from sklearn.mixture import GaussianMixture as GMM
from scipy.stats import spearmanr, kendalltau

In [2]:
x_test = pd.read_csv('data/x_test.csv')
y_test = pd.read_csv('data/y_test.csv')

/home/maxima1ist/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
y_test_arr = []

for value in y_test['hit']:
    if isinstance(value, str) and value.isdigit():
        y_test_arr.append(int(value))
    else:
        y_test_arr.append(np.random.randint(0, 1))

In [4]:
model = GMM(n_components=2, covariance_type='full')
model.fit(x_test)

GaussianMixture(n_components=2)

In [5]:
y_test_predict_proba = [probas[0] for probas in model.predict_proba(x_test)]

In [6]:
valid_test_results = {}
with open('data/valid_test_results.json') as fout:
    valid_test_results = json.load(fout)

In [7]:
list_of_real_tournament_range = []
list_of_predicted_tournament_range = []

curr_i = 0
for tournament_id in valid_test_results:
    tournament = valid_test_results[tournament_id]

    real_tournament_range = []
    predicted_tournament_range = []
    for team in tournament:
        real_mask = team['mask']
        real_count = 0
        for i in range(len(real_mask)):
            if isinstance(real_mask[i], str) and real_mask[i].isdigit():
                real_count += int(real_mask[i])

        predict_proba = [1 for _ in range(len(real_mask))]
        for teammate in team['teamMembers']:
            for question_id in range(len(real_mask)):
                predict_proba[question_id] *= y_test_predict_proba[curr_i]
                curr_i += 1
        predict_proba = [(1 - p) for p in predict_proba]

        real_tournament_range.append((team['team']['id'], real_count / len(real_mask)))
        predicted_tournament_range.append((team['team']['id'], np.prod(predict_proba)))

    list_of_real_tournament_range.append(real_tournament_range)
    list_of_predicted_tournament_range.append(predicted_tournament_range)

In [8]:
spearmanr_coefs = []
kendalltau_coefs = []

for i in range(len(list_of_real_tournament_range)):
    predict_tournament_rating = list_of_predicted_tournament_range[i]
    predict_tournament_rating.sort(key=lambda x:x[1])
    predict_tournament_rating = [pair[0] for pair in predict_tournament_rating]

    real_tournament_rating = list_of_real_tournament_range[i]
    real_tournament_rating.sort(key=lambda x:x[1])
    real_tournament_rating = [pair[0] for pair in real_tournament_rating]
    coef, p = spearmanr(real_tournament_rating, predict_tournament_rating)
    spearmanr_coefs.append(abs(coef))
    coef, p = kendalltau(real_tournament_rating, predict_tournament_rating)
    kendalltau_coefs.append(abs(coef))

In [9]:
spearmanr_coefs

[0.060539740764144694,
 0.18177647170347902,
 0.026060606060606062,
 0.24616665002163854,
 0.030185981854391666,
 0.12791345465524545,
 0.02431169840652934,
 0.3274353225684561,
 0.0027275148390888695,
 0.011472734254992319,
 0.07939482213386871,
 0.13801258675647063,
 0.03131198181649589,
 0.00970282578339936,
 0.16900096679000412,
 0.03383731709724503,
 0.1936274509803922,
 0.11283697082695378,
 0.06816178814262532,
 0.01906362545018007,
 0.11379829287184622,
 0.039605987233105884,
 0.08063906791891136,
 0.030927092709270923,
 0.10621703512512623,
 0.07481695879252734,
 0.014983858767314109,
 0.19217391304347822,
 0.0770224582220656,
 0.05659600614439323,
 0.1322884012539185,
 0.11123349668060363,
 0.14347587191435668,
 0.034482758620689655,
 0.2120300751879699,
 0.1353766067632539,
 0.0961038961038961,
 0.05117628445700325,
 0.07668541903083274,
 0.046537629037629036,
 0.007142857142857145,
 0.0027830786588848187,
 0.4755244755244756,
 0.031198686371100168,
 0.0054766048875032666,
 

In [10]:
kendalltau_coefs

[0.04252269469660774,
 0.12066522285500389,
 0.0006184291898577613,
 0.16867345178540624,
 0.02055800293685756,
 0.08491527432160208,
 0.015286954268687652,
 0.23162623539982033,
 0.0036417884252555838,
 0.0035842293906810036,
 0.055527708159287106,
 0.09178727644267305,
 0.024615320950955574,
 0.004889377826671557,
 0.11077844311377247,
 0.022968004950314858,
 0.14705882352941174,
 0.07392586454804669,
 0.044677027646824415,
 0.023673469387755105,
 0.07650395581430063,
 0.02857142857142857,
 0.05768334731152735,
 0.025050505050505056,
 0.06994759626338574,
 0.05440643332674182,
 0.010893752055881271,
 0.13043478260869565,
 0.0500763116687479,
 0.03767741935483871,
 0.09285159285159285,
 0.07486000589448864,
 0.09364125276811135,
 0.0,
 0.12631578947368421,
 0.08880297985213613,
 0.04761904761904762,
 0.035454103933948516,
 0.0526434394894206,
 0.03182743182743183,
 0.00975609756097561,
 0.00631276901004304,
 0.3636363636363636,
 0.010582010582010581,
 0.004418262150220914,
 0.12053872